<a href="https://colab.research.google.com/github/rajeevak40/Retail_Analytics/blob/main/Customer_Predictive_Analytics_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import pickle

# Data Preparation

In [41]:
# Loading data
df_purchase=pd.read_csv('/content/drive/MyDrive/Retail/purchase data.csv', index_col=0)

# importing pickel files
scaler= pickle.load(open('/content/drive/MyDrive/Retail/scaler.pickle','rb'))
pca= pickle.load(open('/content/drive/MyDrive/Retail/pca.pickle','rb'))
kmean_pca= pickle.load(open('/content/drive/MyDrive/Retail/kmean_pca.pickle','rb'))

# Column Filtering
df_features= df_purchase[['Sex', 'Marital status', 'Age','Education', 'Income', 'Occupation', 'Settlement size']]

# Scaling Data
df_purchase_segm_std=scaler.transform(df_features)


# PCA fitting
df_purchase_segm_pca=pca.transform(df_purchase_segm_std)

# KMeans Clustering
purchase_segm_pca_kmean = kmean_pca.predict(df_purchase_segm_pca)

# Create Purchase Predictor dataset 
df_purchase_predictors= df_purchase.copy()
df_purchase_predictors['Segment']= purchase_segm_pca_kmean

segment_dummies = pd.get_dummies(purchase_segm_pca_kmean, prefix='Segment', prefix_sep='_')
df_purchase_predictors= df_purchase_predictors.reset_index()
df_purchase_predictors=pd.concat([df_purchase_predictors, segment_dummies], axis=1)


In [45]:
df_purchase_predictors.head()


,ID,Day,Incidence,Brand,Quantity,Last_Inc_Brand,Last_Inc_Quantity,Price_1,Price_2,Price_3,Price_4,Price_5,Promotion_1,Promotion_2,Promotion_3,Promotion_4,Promotion_5,Sex,Marital status,Age,Education,Income,Occupation,Settlement size,Segment,Segment_0,Segment_1,Segment_2,Segment_3
0,200000001,1,0,0,0,0,0,1.59,1.87,2.01,2.09,2.66,0,1,0,0,0,0,0,47,1,110866,1,0,2,0,0,1,0
1,200000001,11,0,0,0,0,0,1.51,1.89,1.99,2.09,2.66,0,0,0,0,0,0,0,47,1,110866,1,0,2,0,0,1,0
2,200000001,12,0,0,0,0,0,1.51,1.89,1.99,2.09,2.66,0,0,0,0,0,0,0,47,1,110866,1,0,2,0,0,1,0
3,200000001,16,0,0,0,0,0,1.52,1.89,1.98,2.09,2.66,0,0,0,0,0,0,0,47,1,110866,1,0,2,0,0,1,0
4,200000001,18,0,0,0,0,0,1.52,1.89,1.99,2.09,2.66,0,0,0,0,0,0,0,47,1,110866,1,0,2,0,0,1,0
